In [ ]:
#Install libraries if not available
#pip install requests
#pip install sqlalchemy

In [2]:
#Libraries
import pandas as pd
import requests
import re
from datetime import datetime
from sqlalchemy import create_engine, text
import urllib

#Data Extraction - Extract
#Azure Credentials
azure_server = 'serverName'
azure_database = 'databaseName'
azure_db_username = 'databaseUserName'
azure_db_password = 'databasePassword'

#Set-up connection with Azure Database
driver = '{ODBC Driver 17 for SQL Server}'
odbc_str = f'DRIVER={driver};SERVER={azure_server};PORT=1433;UID={azure_db_username};DATABASE={azure_database};PWD={azure_db_password};Connection Timeout=30'
connect_str = 'mssql+pyodbc:///?odbc_connect=' + urllib.parse.quote_plus(odbc_str)
engine = create_engine(connect_str, echo=True)

#Acuity Scheduling Credentials
acuity_user_id = 'apiUserID'
acuity_api_key = 'apiKey'

#Retrieve Customers' Data from Acuity Scheduling
def retrieve_acuity_data(): 
    acuity_appointments_url = f'https://acuityscheduling.com/api/v1/appointments?user={acuity_user_id}'
    response_appointments = requests.get(acuity_appointments_url, auth=(acuity_user_id, acuity_api_key))
    
    if response_appointments.status_code == 200: 
        raw_appointments_data = response_appointments.json() 
        acuityAppts_df = pd.DataFrame(raw_appointments_data)
        return acuityAppts_df
    else: 
        print("Failed to retrieve data:", response_appointments.status_code, response_appointments.text)
        return None

#Employees File Path
file_path = r'pathName'
sheet_name = 'employees'

#Retrieve Employees Data from Excel in Local Computer
def retrieve_employees_data(file_path, sheet_name): 
    employees_df = pd.read_excel(file_path, sheet_name)
    employees_df.drop_duplicates(inplace=True)
    
    employees_df['startDate'] = pd.to_datetime(employees_df['startDate'], errors='coerce').dt.strftime('%Y-%m-%d')
    employees_df['endDate'] = pd.to_datetime(employees_df['endDate'], errors='coerce').dt.strftime('%Y-%m-%d')
    
    print("Employees data loaded from Excel:\n", employees_df.head())
    return employees_df 

#Data Cleaning - Transform 
def data_cleaning(acuityAppts_df):
    def extract_formsText_details(text): 
        address_match = re.search(r"Address:\s*(.*)", text)
        name_match = re.search(r"Pet's Name\s*:\s*(.*)", text)
        breed_match = re.search(r"Pet's Breed\s*:\s*(.*)", text)
        age_match = re.search(r"Pet's Age\s*:\s*(.*)", text)
        remarks_match = re.search(r"Remarks:\s*(.*)", text)

        return {
            'address': address_match.group(1) if address_match else None,
            'petName': name_match.group(1) if name_match else None,
            'petBreed': breed_match.group(1) if breed_match else None,
            'petAge': age_match.group(1) if age_match else None,
            'remarks': remarks_match.group(1) if remarks_match else None
        }
    
    acuityAppts_df['formsText'] = acuityAppts_df['formsText'].fillna('')
    extracted_formsText_list = acuityAppts_df['formsText'].apply(extract_formsText_details).tolist()
    extracted_details_df = pd.DataFrame(extracted_formsText_list)
    acuityAppts_df = pd.concat([acuityAppts_df, extracted_details_df], axis=1)
    
    type_column_split = acuityAppts_df["type"].str.split(" - ", expand=True)
    acuityAppts_df["serviceType"] = type_column_split[0]
    acuityAppts_df["groomType"] = type_column_split[1]
    
    acuityAppts_df["groomType"] = acuityAppts_df["groomType"].apply(lambda value: re.sub(r'\s*\(.*?\)', '', value) if value else value)
    acuityAppts_df["groomType"] = acuityAppts_df["groomType"].apply(lambda value: re.sub(r'\s*\+.*', '', value) if value else value)
    acuityAppts_df["groomType"] = acuityAppts_df["groomType"].apply(lambda value: value.strip().title() if value else value)
    
    drop_columns = [
        'id','datetimeCreated','datetime',
        'priceSold','amountPaid','appointmentTypeID','addonIDs',
        'classID','calendar','calendarID','certificate','confirmationPage',
        'confirmationPagePaymentLink','location','notes','timezone',
        'calendarTimezone','canClientCancel','canClientReschedule',
        'labels','forms','type','formsText'
    ]
    acuityAppts_df.drop(columns=[col for col in drop_columns if col in acuityAppts_df.columns], inplace=True)

    rename_columns = {
        'date':'apptDate',
        'dateCreated':'salesDate',
        'time':'startTime',
        'price':'salesAmount',
        'paid':'amountPaid',
        'type':'groomType',
        'category':'petType',
        'canceled':'canceledAppt'
    }
    acuityAppts_df.rename(columns=rename_columns, inplace=True)
    
    if 'startTime' in acuityAppts_df.columns:
        acuityAppts_df['startTime'] = acuityAppts_df['startTime'].apply(
            lambda time_str: datetime.strptime(time_str, '%I:%M%p').strftime('%H:%M:%S') if time_str else None
        )
    if 'endTime' in acuityAppts_df.columns:
        acuityAppts_df['endTime'] = acuityAppts_df['endTime'].apply(
            lambda time_str: datetime.strptime(time_str, '%I:%M%p').strftime('%H:%M:%S') if time_str else None
        )

    if 'salesDate' in acuityAppts_df.columns:
        acuityAppts_df['salesDate'] = pd.to_datetime(acuityAppts_df['salesDate'], errors='coerce').dt.strftime('%Y-%m-%d')
    if 'apptDate' in acuityAppts_df.columns:
        acuityAppts_df['apptDate'] = pd.to_datetime(acuityAppts_df['apptDate'], errors='coerce').dt.strftime('%Y-%m-%d')
    
    if 'canceledAppt' in acuityAppts_df.columns:
        acuityAppts_df['canceledAppt'] = acuityAppts_df['canceledAppt'].apply(lambda x: 1 if x else 0)
    if 'amountPaid' in acuityAppts_df.columns:
        acuityAppts_df['amountPaid'] = acuityAppts_df['amountPaid'].apply(lambda x: 1 if x else 0)
    
    for col in acuityAppts_df.select_dtypes(include=['object']).columns:
        acuityAppts_df[col] = acuityAppts_df[col].str.lower().str.strip()
    
    acuityAppts_df['appointment_id'] = acuityAppts_df.index + 1

    acuityAppts_df.drop_duplicates(inplace=True)
    
    print("Unique values in 'canceledAppt' column after conversion:", acuityAppts_df['canceledAppt'].unique())
    print("Data types of the DataFrame columns:\n", acuityAppts_df.dtypes)

    return acuityAppts_df

#Split Dataframe into respective tables 
def split_dataframes(acuityAppts_df, employees_df): 
    address_df = acuityAppts_df[['address']].drop_duplicates()
    services_df = acuityAppts_df[['serviceType', 'groomType']].drop_duplicates()
    customers_df = acuityAppts_df[['firstName', 'lastName', 'email', 'phone', 'address']].drop_duplicates()
    pets_df = acuityAppts_df[['petName', 'petType', 'petAge', 'petBreed', 'remarks', 'email']].drop_duplicates(subset=['email', 'petName'], keep='first')
    appointments_df = acuityAppts_df[['apptDate', 'startTime', 'endTime', 'duration', 'canceledAppt', 'email', 'serviceType', 'groomType', 'appointment_id']].drop_duplicates()
    sales_df = acuityAppts_df[['salesDate', 'salesAmount', 'amountPaid', 'appointment_id']].drop_duplicates()
    employees_staging_df = employees_df[['name', 'job_status', 'role', 'startDate', 'endDate', 'email', 'phone', 'address']].drop_duplicates()
    job_status_df = employees_df[['job_status']].drop_duplicates()
    employee_role_df = employees_df[['role']].drop_duplicates()
    
    return address_df, services_df, customers_df, pets_df, appointments_df, sales_df, employees_staging_df, job_status_df, employee_role_df

#Load
#Database Connection and Check if Table Exist before importing data into staging tables
def transfer_data_to_staging(engine, df, table_name):
    with engine.begin() as connection:
        df.to_sql(table_name, connection, if_exists='replace', index=False)
        print(f"Data transferred to staging table {table_name} successfully.")

#Transfer Data from Staging Tables to Actual SQL Database
def transfer_data_to_actual():
    connection_string = connect_str
    engine = create_engine(connection_string)
    try:
        with engine.begin() as connection:
            # Debug: print out addresses being inserted
            print("Addresses being inserted into the 'address' table:")
            address_debug_query = """
            SELECT DISTINCT address FROM staging_address
            WHERE NOT EXISTS (SELECT 1 FROM address WHERE address = staging_address.address);
            """
            address_debug_result = connection.execute(address_debug_query)
            for row in address_debug_result:
                print(row)

            transfer_queries = [
                """
                INSERT INTO address (address)
                SELECT DISTINCT address FROM staging_address
                WHERE NOT EXISTS (SELECT 1 FROM address WHERE address = staging_address.address);
                """,
                """
                INSERT INTO services (serviceType, groomType)
                SELECT DISTINCT serviceType, groomType FROM staging_services
                WHERE NOT EXISTS (SELECT 1 FROM services WHERE serviceType = staging_services.serviceType AND groomType = staging_services.groomType);
                """,
                """
                INSERT INTO job_status (job_status)
                SELECT DISTINCT job_status FROM staging_job_status
                WHERE NOT EXISTS (SELECT 1 FROM job_status WHERE job_status = staging_job_status.job_status);
                """,
                """
                INSERT INTO employee_role (role)
                SELECT DISTINCT role FROM staging_employee_role
                WHERE NOT EXISTS (SELECT 1 FROM employee_role WHERE role = staging_employee_role.role);
                """,
                """
                MERGE INTO customers AS target
                USING (
                    SELECT sc.firstName, sc.lastName, sc.email, sc.phone, a.address_id
                    FROM (
                        SELECT *, ROW_NUMBER() OVER(PARTITION BY email ORDER BY email) AS rn
                        FROM staging_customers
                    ) sc
                    JOIN address a ON sc.address = a.address
                    WHERE sc.rn = 1
                ) AS source
                ON (target.email = source.email)
                WHEN MATCHED THEN
                    UPDATE SET 
                        target.firstName = source.firstName,
                        target.lastName = source.lastName,
                        target.phone = source.phone,
                        target.address_id = source.address_id
                WHEN NOT MATCHED THEN
                    INSERT (firstName, lastName, email, phone, address_id)
                    VALUES (source.firstName, source.lastName, source.email, source.phone, source.address_id);
                """,
                """
                INSERT INTO pets (customer_id, petName, petType, petAge, petBreed, remarks)
                SELECT c.customer_id, sp.petName, sp.petType, sp.petAge, sp.petBreed, sp.remarks
                FROM (
                    SELECT email, petName, petType, petAge, petBreed, remarks,
                    ROW_NUMBER() OVER(PARTITION BY email, petName ORDER BY email, petName) AS rn
                    FROM staging_pets
                ) sp
                JOIN customers c ON sp.email = c.email
                WHERE sp.rn = 1
                AND NOT EXISTS (
                    SELECT 1 FROM pets p WHERE p.customer_id = c.customer_id AND p.petName = sp.petName
                );
                """,
                """
                SET IDENTITY_INSERT appointments ON;

                INSERT INTO appointments (customer_id, service_id, apptDate, startTime, endTime, duration, canceledAppt, appointment_id)
                SELECT c.customer_id, s.service_id, sa.apptDate, sa.startTime, sa.endTime, sa.duration, sa.canceledAppt, sa.appointment_id
                FROM staging_appointments sa
                JOIN customers c ON sa.email = c.email
                JOIN services s ON sa.serviceType = s.serviceType AND sa.groomType = s.groomType
                WHERE NOT EXISTS (SELECT 1 FROM appointments a WHERE a.appointment_id = sa.appointment_id);

                SET IDENTITY_INSERT appointments OFF;
                """,
                """
                INSERT INTO sales (salesDate, salesAmount, amountPaid, appointment_id)
                SELECT ss.salesDate, ss.salesAmount, ss.amountPaid, sa.appointment_id
                FROM staging_sales ss
                JOIN staging_appointments sa ON ss.appointment_id = sa.appointment_id
                WHERE NOT EXISTS (SELECT 1 FROM sales s WHERE s.appointment_id = sa.appointment_id);
                """,
                # Insert employee addresses before inserting employees
                """
                INSERT INTO address (address)
                SELECT DISTINCT se.address
                FROM staging_employees se
                LEFT JOIN address a ON se.address = a.address
                WHERE a.address_id IS NULL;
                """,
                """
                INSERT INTO employees (job_status_id, role_id, address_id, name, startDate, endDate, email, phone)
                SELECT js.job_status_id, er.role_id, a.address_id, se.name, se.startDate, se.endDate, se.email, se.phone
                FROM (
                    SELECT DISTINCT name, job_status, role, startDate, endDate, email, phone, address,
                    ROW_NUMBER() OVER(PARTITION BY email ORDER BY email) AS rn
                    FROM staging_employees
                ) se
                JOIN job_status js ON se.job_status = js.job_status
                JOIN employee_role er ON se.role = er.role
                JOIN address a ON se.address = a.address
                WHERE se.rn = 1
                AND NOT EXISTS (
                    SELECT 1 FROM employees e WHERE e.email = se.email
                );
                """
            ]

            for query in transfer_queries:
                print(f"Executing query:\n{query}\n")
                result = connection.execute(text(query))
                print(f"Rows affected: {result.rowcount}")

            debug_employee_insertion(connection)
            final_tables = ['address', 'services', 'job_status', 'employee_role', 'customers', 'pets', 'appointments', 'sales', 'employees']
            for table in final_tables:
                verify_transfer(connection, table)

            print("Data transferred to actual tables successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")

#Debug for Employees' Data import due to error 
def debug_employee_insertion(connection):
    query = """
    SELECT se.name, se.job_status, se.role, se.startDate, se.endDate, se.email, se.phone, se.address, 
           js.job_status_id, er.role_id, a.address_id
    FROM staging_employees se
    LEFT JOIN job_status js ON se.job_status = js.job_status
    LEFT JOIN employee_role er ON se.role = er.role
    LEFT JOIN address a ON se.address = a.address;
    """
    result = connection.execute(query)
    employee_debug_df = pd.DataFrame(result.fetchall(), columns=result.keys())
    print("Employee Debug Data:\n", employee_debug_df)

def verify_transfer(connection, table_name):
    query = f"SELECT COUNT(*) FROM {table_name}"
    result = connection.execute(query).fetchone()
    print(f"Number of records in {table_name}: {result[0]}")
    sample_query = f"SELECT TOP 10 * FROM {table_name}"
    sample_result = connection.execute(sample_query)
    sample_df = pd.DataFrame(sample_result.fetchall(), columns=sample_result.keys())
    print(f"Sample data from {table_name}:\n", sample_df)

#Main Execution
def main():
    acuityAppts_df = retrieve_acuity_data()
    employees_df = retrieve_employees_data(file_path, sheet_name)
    
    if acuityAppts_df is not None:
        acuityAppts_df = data_cleaning(acuityAppts_df)
        address_df, services_df, customers_df, pets_df, appointments_df, sales_df, employees_staging_df, job_status_df, employee_role_df = split_dataframes(acuityAppts_df, employees_df)
        
        #Transfer data to staging
        transfer_data_to_staging(engine, address_df, 'staging_address')
        transfer_data_to_staging(engine, services_df, 'staging_services')
        transfer_data_to_staging(engine, customers_df, 'staging_customers')
        transfer_data_to_staging(engine, pets_df, 'staging_pets')
        transfer_data_to_staging(engine, appointments_df, 'staging_appointments')
        transfer_data_to_staging(engine, sales_df, 'staging_sales')
        transfer_data_to_staging(engine, employees_staging_df, 'staging_employees')
        transfer_data_to_staging(engine, job_status_df, 'staging_job_status')
        transfer_data_to_staging(engine, employee_role_df, 'staging_employee_role')

        #Transfer data to actual SQL
        transfer_data_to_actual()
    else:
        print("No data retrieved from Acuity Scheduling.")

if __name__ == "__main__":
    main()

Employees data loaded from Excel:
        name job_status            role   startDate endDate  \
0  Dona Loh     Active    Head Stylist  2017-01-21     NaN   
1     Hafiz     Active  Feline Stylist  2019-12-15     NaN   

                 email     phone  \
0    nana123@gmail.com  12345678   
1  hello88@hotmail.com  98765432   

                                             address  
0       21 jurong east st 88 #22-11 singapore 240281  
1  88 Bukit Batok East Avenue 9 #09-31 singapore ...  
Unique values in 'canceledAppt' column after conversion: [0]
Data types of the DataFrame columns:
 firstName         object
lastName          object
phone             object
email             object
apptDate          object
startTime         object
endTime           object
salesDate         object
salesAmount       object
amountPaid         int64
petType           object
duration          object
canceledAppt       int64
address           object
petName           object
petBreed          object
petAg

2024-06-20 09:16:51,294 INFO sqlalchemy.engine.Engine [generated in 0.00136s] ('staging_address', 'dbo')
2024-06-20 09:16:52,160 INFO sqlalchemy.engine.Engine select ind.index_id, ind.is_unique, ind.name, ind.filter_definition from sys.indexes as ind join sys.tables as tab on ind.object_id=tab.object_id join sys.schemas as sch on sch.schema_id=tab.schema_id where tab.name = CAST(? AS NVARCHAR(max)) and sch.name=CAST(? AS NVARCHAR(max)) and ind.is_primary_key=0 and ind.type != 0
2024-06-20 09:16:52,162 INFO sqlalchemy.engine.Engine [generated in 0.00139s] ('staging_address', 'dbo')
2024-06-20 09:16:52,213 INFO sqlalchemy.engine.Engine select ind_col.index_id, ind_col.object_id, col.name, ind_col.is_included_column from sys.columns as col join sys.tables as tab on tab.object_id=col.object_id join sys.index_columns as ind_col on (ind_col.column_id=col.column_id and ind_col.object_id=tab.object_id) join sys.schemas as sch on sch.schema_id=tab.schema_id where tab.name=CAST(? AS NVARCHAR(max

2024-06-20 09:16:52,907 INFO sqlalchemy.engine.Engine [cached since 1.615s ago] ('staging_services', 'dbo')
2024-06-20 09:16:53,491 INFO sqlalchemy.engine.Engine select ind.index_id, ind.is_unique, ind.name, ind.filter_definition from sys.indexes as ind join sys.tables as tab on ind.object_id=tab.object_id join sys.schemas as sch on sch.schema_id=tab.schema_id where tab.name = CAST(? AS NVARCHAR(max)) and sch.name=CAST(? AS NVARCHAR(max)) and ind.is_primary_key=0 and ind.type != 0
2024-06-20 09:16:53,492 INFO sqlalchemy.engine.Engine [cached since 1.332s ago] ('staging_services', 'dbo')
2024-06-20 09:16:53,542 INFO sqlalchemy.engine.Engine select ind_col.index_id, ind_col.object_id, col.name, ind_col.is_included_column from sys.columns as col join sys.tables as tab on tab.object_id=col.object_id join sys.index_columns as ind_col on (ind_col.column_id=col.column_id and ind_col.object_id=tab.object_id) join sys.schemas as sch on sch.schema_id=tab.schema_id where tab.name=CAST(? AS NVARCH

2024-06-20 09:16:54,094 INFO sqlalchemy.engine.Engine [cached since 2.802s ago] ('staging_customers', 'dbo')
2024-06-20 09:16:54,726 INFO sqlalchemy.engine.Engine select ind.index_id, ind.is_unique, ind.name, ind.filter_definition from sys.indexes as ind join sys.tables as tab on ind.object_id=tab.object_id join sys.schemas as sch on sch.schema_id=tab.schema_id where tab.name = CAST(? AS NVARCHAR(max)) and sch.name=CAST(? AS NVARCHAR(max)) and ind.is_primary_key=0 and ind.type != 0
2024-06-20 09:16:54,727 INFO sqlalchemy.engine.Engine [cached since 2.567s ago] ('staging_customers', 'dbo')
2024-06-20 09:16:54,780 INFO sqlalchemy.engine.Engine select ind_col.index_id, ind_col.object_id, col.name, ind_col.is_included_column from sys.columns as col join sys.tables as tab on tab.object_id=col.object_id join sys.index_columns as ind_col on (ind_col.column_id=col.column_id and ind_col.object_id=tab.object_id) join sys.schemas as sch on sch.schema_id=tab.schema_id where tab.name=CAST(? AS NVAR

2024-06-20 09:16:55,521 INFO sqlalchemy.engine.Engine [cached since 4.229s ago] ('staging_pets', 'dbo')
2024-06-20 09:16:56,238 INFO sqlalchemy.engine.Engine select ind.index_id, ind.is_unique, ind.name, ind.filter_definition from sys.indexes as ind join sys.tables as tab on ind.object_id=tab.object_id join sys.schemas as sch on sch.schema_id=tab.schema_id where tab.name = CAST(? AS NVARCHAR(max)) and sch.name=CAST(? AS NVARCHAR(max)) and ind.is_primary_key=0 and ind.type != 0
2024-06-20 09:16:56,240 INFO sqlalchemy.engine.Engine [cached since 4.079s ago] ('staging_pets', 'dbo')
2024-06-20 09:16:56,307 INFO sqlalchemy.engine.Engine select ind_col.index_id, ind_col.object_id, col.name, ind_col.is_included_column from sys.columns as col join sys.tables as tab on tab.object_id=col.object_id join sys.index_columns as ind_col on (ind_col.column_id=col.column_id and ind_col.object_id=tab.object_id) join sys.schemas as sch on sch.schema_id=tab.schema_id where tab.name=CAST(? AS NVARCHAR(max))

...

2024-06-20 09:16:59,231 INFO sqlalchemy.engine.Engine [cached since 7.938s ago] ('staging_sales', 'dbo')
2024-06-20 09:16:59,951 INFO sqlalchemy.engine.Engine select ind.index_id, ind.is_unique, ind.name, ind.filter_definition from sys.indexes as ind join sys.tables as tab on ind.object_id=tab.object_id join sys.schemas as sch on sch.schema_id=tab.schema_id where tab.name = CAST(? AS NVARCHAR(max)) and sch.name=CAST(? AS NVARCHAR(max)) and ind.is_primary_key=0 and ind.type != 0
2024-06-20 09:16:59,953 INFO sqlalchemy.engine.Engine [cached since 7.793s ago] ('staging_sales', 'dbo')
2024-06-20 09:17:00,131 INFO sqlalchemy.engine.Engine select ind_col.index_id, ind_col.object_id, col.name, ind_col.is_included_column from sys.columns as col join sys.tables as tab on tab.object_id=col.object_id join sys.index_columns as ind_col on (ind_col.column_id=col.column_id and ind_col.object_id=tab.object_id) join sys.schemas as sch on sch.schema_id=tab.schema_id where tab.name=CAST(? AS NVARCHAR(max

2024-06-20 09:17:01,851 INFO sqlalchemy.engine.Engine [cached since 10.56s ago] ('staging_employees', 'dbo')
2024-06-20 09:17:02,410 INFO sqlalchemy.engine.Engine select ind.index_id, ind.is_unique, ind.name, ind.filter_definition from sys.indexes as ind join sys.tables as tab on ind.object_id=tab.object_id join sys.schemas as sch on sch.schema_id=tab.schema_id where tab.name = CAST(? AS NVARCHAR(max)) and sch.name=CAST(? AS NVARCHAR(max)) and ind.is_primary_key=0 and ind.type != 0
2024-06-20 09:17:02,411 INFO sqlalchemy.engine.Engine [cached since 10.25s ago] ('staging_employees', 'dbo')
2024-06-20 09:17:02,433 INFO sqlalchemy.engine.Engine select ind_col.index_id, ind_col.object_id, col.name, ind_col.is_included_column from sys.columns as col join sys.tables as tab on tab.object_id=col.object_id join sys.index_columns as ind_col on (ind_col.column_id=col.column_id and ind_col.object_id=tab.object_id) join sys.schemas as sch on sch.schema_id=tab.schema_id where tab.name=CAST(? AS NVAR

2024-06-20 09:17:03,627 INFO sqlalchemy.engine.Engine [cached since 12.34s ago] ('staging_job_status', 'dbo')
2024-06-20 09:17:04,414 INFO sqlalchemy.engine.Engine select ind.index_id, ind.is_unique, ind.name, ind.filter_definition from sys.indexes as ind join sys.tables as tab on ind.object_id=tab.object_id join sys.schemas as sch on sch.schema_id=tab.schema_id where tab.name = CAST(? AS NVARCHAR(max)) and sch.name=CAST(? AS NVARCHAR(max)) and ind.is_primary_key=0 and ind.type != 0
2024-06-20 09:17:04,415 INFO sqlalchemy.engine.Engine [cached since 12.25s ago] ('staging_job_status', 'dbo')
2024-06-20 09:17:04,476 INFO sqlalchemy.engine.Engine select ind_col.index_id, ind_col.object_id, col.name, ind_col.is_included_column from sys.columns as col join sys.tables as tab on tab.object_id=col.object_id join sys.index_columns as ind_col on (ind_col.column_id=col.column_id and ind_col.object_id=tab.object_id) join sys.schemas as sch on sch.schema_id=tab.schema_id where tab.name=CAST(? AS NV

2024-06-20 09:17:04,904 INFO sqlalchemy.engine.Engine [cached since 13.61s ago] ('staging_employee_role', 'dbo')
2024-06-20 09:17:05,600 INFO sqlalchemy.engine.Engine select ind.index_id, ind.is_unique, ind.name, ind.filter_definition from sys.indexes as ind join sys.tables as tab on ind.object_id=tab.object_id join sys.schemas as sch on sch.schema_id=tab.schema_id where tab.name = CAST(? AS NVARCHAR(max)) and sch.name=CAST(? AS NVARCHAR(max)) and ind.is_primary_key=0 and ind.type != 0
2024-06-20 09:17:05,601 INFO sqlalchemy.engine.Engine [cached since 13.44s ago] ('staging_employee_role', 'dbo')
2024-06-20 09:17:05,652 INFO sqlalchemy.engine.Engine select ind_col.index_id, ind_col.object_id, col.name, ind_col.is_included_column from sys.columns as col join sys.tables as tab on tab.object_id=col.object_id join sys.index_columns as ind_col on (ind_col.column_id=col.column_id and ind_col.object_id=tab.object_id) join sys.schemas as sch on sch.schema_id=tab.schema_id where tab.name=CAST(?

Sample data from employee_role:
    role_id            role
0        5  Feline Stylist
1        6    Head Stylist
Number of records in customers: 26
Sample data from customers:
    customer_id firstName     lastName                         email     phone  \
0          391  ambrosio       sipson             asipson12@ask.com  93151001   
1          392      bord      benning             bbenning6@mlb.com  52634921   
2          393     brady      gamblin            bgamblin8@lulu.com  74585201   
3          394    carmon       alcock            calcock1v@gmpg.org  23895036   
4          395   cecilia    dalgliesh     cdalgliesh2@wordpress.com  74150321   
5          396     celie      watsham        cwatshamk@engadget.com  06676143   
6          397     erica   bassingham        ebassingham4@cam.ac.uk  45852369   
7          398     ernst      layzell         elayzell2q@dion.ne.jp  76948003   
8          399    gianni     roglieri  groglieri0@blogtalkradio.com  52013693   
9          4